Implement your own index to take the place of elasticsearch in the HW1 code, and index the document collection used for HW1. Your index should be able to handle large numbers of documents and terms without using excessive memory or disk I/O.

This involves writing two programs:

A tokenizer and indexer
An updated version of your HW1 ranker which uses your inverted index

In [1]:
import os
import nltk
import numpy as np
import pandas as pd
import re
import time
import math
import pickle
from nltk.stem import PorterStemmer
from elasticsearch import Elasticsearch 
global stop_words_removed_stemmed, flatted_dict


In [2]:

## Initialize stemmer

ps = PorterStemmer()

es = Elasticsearch()

tokenize = nltk.word_tokenize

with open (r"C:\Users\mm199\IR-hw\HW2_data\AP_DATA\stoplist.txt","r") as file:
    file  = file.readlines()
stop_words = [i[:-1] for i in file]
stop_words_from_nltk = nltk.corpus.stopwords.words("english")
stop_words_list = list(set(stop_words_from_nltk + stop_words))

pattern =  "\w+(\.?\w+)*"

In [3]:
with open(r"C:\Users\mm199\IR-hw\HW2_data\temp_data\dictfile","rb") as file:
    doc_id_list_mapping_all = pickle.load(file)

In [3]:
## Read the data from all sources
def get_data():
## Get all the docs from the file location
    file_all = os.getcwd() + "\AP_DATA\\ap89_collection"

    ## Remove the read me part
    list_of_all = os.listdir(file_all)[:-1]

    total_doc = 0
    doc_id_list_mapping = {}
    for doc in list_of_all:
        with open(r".\AP_DATA\ap89_collection\\" + doc, "r") as file_data:
            file_data_read = file_data.read().split("<DOC>")
        for each_file in file_data_read:
            if "</DOCNO>" in each_file:
                doc_id_content = each_file.split("</DOCNO>")
                total_doc += 1
                doc_id = doc_id_content[0].strip("\n<DOCNO>").strip()
                doc_content = doc_id_content[1].split("<TEXT>")[1].strip("\n</TEXT>\n</DOC>\n").strip()
                doc_id_list_mapping[total_doc]= [doc_id, doc_content]
    return doc_id_list_mapping

In [22]:
## Read queries

with open(r"C:\Users\mm199\IR-hw\HW1_data\AP_DATA\query_desc.51-100.short.txt", "r") as query:
    query_list = query.readlines()
    
## define the common words from queries
common_words = ['document', 'discuss', 'exist','determine', 'current','pay','even','taken','type', 'report', 'describe', 'cite', 'include', 'identify', 'make', 'one', 'must', 'second', 'use', 'side', 'take', 'predict']

## get all the words in a query
all_query_in_a_list = {i[:5].split(".")[0]:[match.group().lower() for match in re.finditer(pattern,i[3:],re.M|re.I)] for i in query_list}

## words of dictionary which are not in stop words or common words
words_list = {}
for key in all_query_in_a_list:
    if all_query_in_a_list[key] != []:
        words_list[key] = [ps.stem(i) for i in all_query_in_a_list[key] if i not in stop_words_list and i not in common_words]



In [5]:
## call the function to get the data
doc_id_list_mapping = get_data()

In [ ]:
#  dump the file using pickle
def dump(filename, data):
    with open(filename,"wb") as file:
        pickle.dump(data, file)
    return

In [8]:
#  load the file using pickle
def load(filename):
    with open(filename,"rb") as file:
        data = pickle.load(file)
    return data

In [6]:
## change tokenizer refer assignment and remove extra stop words
doc_id_list_mapping_all = {i: [match.group() for match in re.finditer(pattern,doc_id_list_mapping[i][1],re.M|re.I)] for i in doc_id_list_mapping}
print ("tokenizing completed")
#doc_id_list_mapping = {i: doc_id_list_mapping[i][0] for i in doc_id_list_mapping}
print ("doc_id_list_mapping reduction completed")
doc_id_list_mapping_all = {i: list(map(str.lower,doc_id_list_mapping_all[i])) for i in doc_id_list_mapping_all}
print ("lower function converstion completed")
# dump("doc_id_list_mapping_all",doc_id_list_mapping_all)



tokenizing completed
doc_id_list_mapping reduction completed
lower function converstion completed


In [9]:
stop_words_removed_stemmed = {i : [ps.stem(j) for j in doc_id_list_mapping_all[i] if j not in stop_words_list and j not in string.punctuation] for i in doc_id_list_mapping_all}
print ("stop_words_removed_stemmed completed")
# del(doc_id_list_mapping_all)
print ("stemmming completed")


stop_words_removed_stemmed completed
stemmming completed


In [146]:
stop_words_removed_stemmed1 = {i : [j for j in stop_words_removed_stemmed[i] if not re.fullmatch("[\w]{,2}",j)] for i in stop_words_removed_stemmed}

In [27]:
## Define and initialize pandas dataframe to save term freq, doc freq and ttf
total_doc = len(stop_words_removed_stemmed)
list_of_values = list(words_list.values())
list_single_item = list(set([i for item in list_of_values for i in item]))

df_term_freq = pd.DataFrame(data = 0, index = range(1,total_doc+1), columns = list_single_item)
df_ttf_freq = pd.DataFrame(data = 0, index = ["ttf"], columns = list_single_item)
df_doc_freq = pd.DataFrame(data = 0, index = ["doc_freq"], columns = list_single_item)
len_of_doc = pd.DataFrame(data = 0, index = range(1,total_doc+1), columns = ["length"])


In [10]:
## Flatten the list
flat_list = [j for i in stop_words_removed_stemmed for j in stop_words_removed_stemmed[i]]
flatted = list(set(flat_list))
flatted_dict = {word : index+1 for index,word in enumerate(flatted)}
len(flatted)

154634

In [28]:
## Before saving the data in the file, the term inverted doc must be created

def term_wise_dictionary (start, end):  
    global stop_words_removed_stemmed, flatted_dict
    stop_words_removed_stemmed_batchwise = list(stop_words_removed_stemmed.keys())[start:end]
    stitch_dict = {flatted_dict[i]:[] for i in flatted_dict}
    for doc_id in stop_words_removed_stemmed_batchwise:
        pos_dict = {}
        len_of_doc["length"][doc_id] = len(stop_words_removed_stemmed[doc_id])
        for index,word in enumerate(stop_words_removed_stemmed[doc_id]):
            term_id = flatted_dict[word]
            if term_id not in pos_dict.keys():
                pos_dict[term_id] = []
            pos_dict[term_id].append(index + 1)
        for term_id in pos_dict:
            doclist = [doc_id, len(pos_dict[term_id])]
            doclist.extend(pos_dict[term_id])
            stitch_dict[term_id].append(doclist)
    stitch_dict = {i:stitch_dict[i] for i in stitch_dict if stitch_dict[i] != []}
    return stitch_dict
         

Step Two: Indexing
The next step is to record each document’s tokens in an inverted index. The inverted list for a term must contain the following information:

The DF and CF (aka TTF) of the term.
A list of IDs of the documents which contain the term, along with the TF of the term within that document and a list of positions within the document where the term occurs. (The first term in a document has position 1, the second term has position 2, etc.)
You should also store the following information.

The total number of distinct terms (the vocabulary size) and the total number of tokens (total CF) in the document collection.
The map between terms and their IDs, if required by your design.
The map between document names and their IDs, if required by your design.
All inverted lists/files written on the hard drive have to be sorted on DocBlocks by the TF count. This will facilitate merging, in particular with mergesort. 

In [29]:
## Write the data in a file 

def write_into_file(filename, term_dict, seek_num = False):
    with open(filename, "a+") as file:
        catalog = {}
        for term_id in term_dict:
            if seek_num:
                file.seek(seek_num)
                start_offset = seek_num + 1
            else:
                start_offset = file.tell()
            save_in_the_file = str(term_id) + ":"
            for i in term_dict[term_id]:
                temp = str(i).split(" ")
                save_in_the_file += str("".join(temp)).strip("\[").strip("\]")  
                save_in_the_file += ";"
            file.write(save_in_the_file)
            catalog[term_id] = [start_offset, len(save_in_the_file)]
    return catalog
            
        


In [30]:
## Write on file batch wise

i = 0
j = 0
t1 = time.time()
while (i < total_doc):
    t1 = time.time()
    term_dict = term_wise_dictionary(i, i + 1001)
    filename_term = ".\data5\inverted_dict" + str(j+1) + ".txt"
    catalog = write_into_file(filename_term, term_dict)
    filename =  ".\data5\catalog" + str(j+1)
    with open(filename,"wb") as file:
        pickle.dump(catalog, file)
    del(term_dict,catalog)
    i += 1000
    j += 1
    if j % 10 == 0:
        print("Completed iteration", j, (time.time() - t1))
        t1 = time.time()

    

    

Completed iteration 10 3.587498903274536
Completed iteration 20 3.5397298336029053
Completed iteration 30 5.2720301151275635
Completed iteration 40 3.805634021759033
Completed iteration 50 3.7087371349334717
Completed iteration 60 3.6201822757720947
Completed iteration 70 3.16208553314209
Completed iteration 80 3.8442792892456055


In [31]:
def open_file_and_read_data(term_id, filename, catalog):
    start_offset_position = 0
    len_position_after_offset = 1
    value = catalog[term_id][start_offset_position]
    with open(filename, "r") as file:
        file.seek(value)
        row_reader = file.read(catalog[term_id][len_position_after_offset])
    read = parser_returns_a_list(row_reader)
    return read


In [32]:
def parser_returns_a_list(read):
    new_dict = {}
    for index, i in enumerate(read):
        if i == ":":
            semi_colon = index
            new_dict[int(read[:semi_colon])] = []
            list_to_match = re.split(";", read[semi_colon+1:-1])
            for i in list_to_match:
                if i:
                    each_list = re.split(",", i)
                    each_list = [int(i) for i in each_list]
                    new_dict[int(read[:semi_colon])].append(each_list)
    return new_dict


In [33]:
def merger_func(filename_catalog1, filename1, filename_catalog2, filename2):
    t1 = time.time()
    catalog1 = filename_catalog1  
    with open(filename_catalog2,"rb") as file:
        catalog2 = pickle.load(file)
    recently_seek_added_address = False
    new_filename = ".\data5\\"+ str(int(t1)) + ".txt"
    new_catalog = {}
    term_dict_temp = {}
    for term_id in catalog1:
        read1 = open_file_and_read_data(term_id, filename1, catalog1)
        if term_id in catalog2:
            read2 = open_file_and_read_data(term_id, filename2, catalog2)
            read_merger = read1[term_id] + read2[term_id]
            term_dict_temp[term_id] = read_merger
            catalog2.pop(term_id)
        else:
            term_dict_temp[term_id] = read1[term_id]
    del (catalog1)   

    for term_id in catalog2:
        read2 = open_file_and_read_data(term_id, filename2, catalog2)
        term_dict_temp[term_id] = read2[term_id]
    del (catalog2)

    new_catalog = write_into_file(new_filename, term_dict_temp, recently_seek_added_address)

    new_catalog_filename =  ".\data5\\"+ str(int(t1)) + "-catalog"
    return new_catalog, new_filename
    

In [36]:
## Call merger on first two batches then repeatedly on all

num_of_files = 85

t1 = time.time()
filename_catalog1 = ".\data5\catalog1" 
filename_catalog2 = ".\data5\catalog2" 

filename1 = ".\data5\inverted_dict1" + ".txt"
filename2 = ".\data5\inverted_dict2" + ".txt"

with open(filename_catalog1,"rb") as file:
    catalog1 = pickle.load(file)   

new_catalog, new_filename = merger_func(catalog1, filename1, filename_catalog2, filename2)
print ("time taken:", time.time() - t1)

t1 = time.time()
i = 3
while (i <= num_of_files):
    filename_catalog = ".\data5\catalog"+ str(i)
    filename = ".\data5\inverted_dict" + str(i) + ".txt"
    prev_file = "%s" %new_filename
    prev_catalog = new_catalog
    new_catalog, new_filename = merger_func(prev_catalog, prev_file, filename_catalog, filename)
    os.remove(prev_file)
    i += 1
    if i % 5 == 0:
        t2 = time.time()
        print ("Iteration completed: ",i, (t2 - t1))
        t1 = t2



time taken: 10.898981809616089
Iteration completed:  5 30.60238289833069
Iteration completed:  10 180.18739914894104
Iteration completed:  15 258.83012223243713
Iteration completed:  20 326.6536662578583
Iteration completed:  25 396.365051984787
Iteration completed:  30 469.6460771560669
Iteration completed:  35 538.0184469223022
Iteration completed:  40 592.7731511592865
Iteration completed:  45 629.7057461738586
Iteration completed:  50 700.815577507019
Iteration completed:  55 776.6601355075836
Iteration completed:  60 852.3921411037445
Iteration completed:  65 934.3486547470093
Iteration completed:  70 1013.6319983005524
Iteration completed:  75 1040.3381969928741
Iteration completed:  80 1075.0428161621094
Iteration completed:  85 680.6833901405334


In [37]:
## Search es for retrieving doc for each term in a query

dict_ids = {}
for key in words_list:
    list_id = []
    t1 = time.time()
    for word in words_list[key]:
        try:
            term_id = flatted_dict[word]
        except KeyError:
            continue
        val = open_file_and_read_data(term_id = term_id, filename = new_filename, catalog = new_catalog)
        list_of_docs_term_present = list(val.values())
        list_of_docs_term_present_3000 = sorted(list_of_docs_term_present[0], key = lambda x:x[1],reverse = True)[:2000] 
        df_doc_freq[word]["doc_freq"] = len(list_of_docs_term_present[0])
        list_of_docs_term_present_ids = [i[0] for i in list_of_docs_term_present_3000]
        for index,doc_id in enumerate(list_of_docs_term_present_ids):
            df_term_freq[word][doc_id] = list_of_docs_term_present_3000[index][1]
        df_ttf_freq[word]["ttf"] = sum([i[1] for i in list_of_docs_term_present[0]])
        list_id.extend(list_of_docs_term_present_ids)
    dict_ids[key] = list(set(list_id))
    print ("Word completed:", key," time taken : ", (time.time() - t1), " for ids ", len(dict_ids[key]))
    

Word completed: 85  time taken :  18.05122685432434  for ids  11016
Word completed: 59  time taken :  19.805733680725098  for ids  12179
Word completed: 56  time taken :  22.053888082504272  for ids  9772
Word completed: 71  time taken :  39.395343542099  for ids  16549
Word completed: 64  time taken :  15.598857164382935  for ids  10034
Word completed: 62  time taken :  13.129813194274902  for ids  8180
Word completed: 93  time taken :  13.471335649490356  for ids  8636
Word completed: 99  time taken :  10.044128894805908  for ids  6261
Word completed: 58  time taken :  11.896251678466797  for ids  4751
Word completed: 77  time taken :  3.5612759590148926  for ids  2088
Word completed: 54  time taken :  26.902849912643433  for ids  15022
Word completed: 87  time taken :  23.602196216583252  for ids  12137
Word completed: 94  time taken :  10.822185754776001  for ids  5997
Word completed: 100  time taken :  34.541754961013794  for ids  18039
Word completed: 89  time taken :  17.9459259

In [38]:
## Initialize other variables

average_doc_length = sum([len(stop_words_removed_stemmed[i]) for i in stop_words_removed_stemmed])/len(stop_words_removed_stemmed)
vocab_size = len(flatted_dict)

global average_doc_length
global total_doc
global vocab_size
global dict_ids
global df_term_freq
global df_ttf_freq
global df_doc_freq
global len_of_doc

In [55]:
## Define all model 

def okapi_model (word_term_freq_list, doc_id):
    okapi = 0.0
    for key in word_term_freq_list:
        tfwd = df_term_freq[key][doc_id]
        okapi += round((tfwd / (tfwd + 0.5 + 1.5 * (len_of_doc["length"][doc_id] / average_doc_length))),4)
    return okapi
    
def tfidf_model(word_term_freq_list, doc_id):  
    tfidf = 0.0
    for key in word_term_freq_list:
        tfwd = df_term_freq[key][doc_id]
        doc_freq = df_doc_freq[key]["doc_freq"]
        if doc_freq == 0:
            tfidf_val = 0
        else:
            tfidf_val = math.log(total_doc / doc_freq)
        tfidf += round(((tfwd / (tfwd + 0.5 + 1.5 * (len_of_doc["length"][doc_id] / average_doc_length))) * tfidf_val),4)
    return tfidf

def okapi_bm25(word_term_freq_list, doc_id):  
    bm25 = 0.0
    tfwq = {}
    k1 = 1.2
    k2 = 500
    b = 0.75
    for key in word_term_freq_list:
        tfwd = df_term_freq[key][doc_id]
        tfwq[key] = word_term_freq_list.count(key)
        bm25 += round(((math.log((total_doc + 0.5) / (df_doc_freq[key]["doc_freq"] + 0.5)))*((tfwd + k1 * tfwd) / (tfwd + k1 * ((1 - b) + b * (len_of_doc["length"][doc_id] / average_doc_length))))*((tfwq[key] + k2 * tfwq[key]) / (tfwq[key] + k2))), 5)
    return bm25 

def laplace_model(word_term_freq_list, doc_id):  
    laplace = 0.0
    for key in word_term_freq_list:
        tfwd = df_term_freq[key][doc_id]
        prob = ((tfwd + 1) / (len_of_doc["length"][doc_id] + (vocab_size)))
        laplace += math.log(prob)
    return laplace

def jm_model(word_term_freq_list, doc_id):
    jm = 0.0
    epsilon = 0.00000001
    lamda = 0.6
    for key in word_term_freq_list:
        tfwd = df_term_freq[key][doc_id]
        ttf = df_ttf_freq[key]["ttf"]
        if tfwd == 0:
            tfwd = epsilon
            ttf = epsilon
        prob_lm = lamda * (tfwd / len_of_doc["length"][doc_id]) + (1 - lamda) * (ttf / vocab_size)
        jm += math.log(prob_lm)
    return jm


In [57]:
## Function for each model per query

def call_each_model(model_name):
    store_result_for_each_query = {}
    for key in words_list:
        result_dict = {}
        for doc_id in dict_ids[key]:
            result_dict[doc_id] = model_name(words_list[key], doc_id)
        store_result_for_each_query[key] = sorted(result_dict.items(), key = lambda x:x[1], reverse = True)[:1000]
    return store_result_for_each_query


In [42]:
## Call each model

okapi_result = call_each_model(okapi_model)
tfidf_result = call_each_model(tfidf_model)
okapi_bm25_result = call_each_model(okapi_bm25)
laplace_result = call_each_model(laplace_model)
jm_result = call_each_model(jm_model)


In [49]:
## Function to save the result in a file

# <query-number> Q0 <docno> <rank> <score> Exp

def save_result (store_result_for_each_query, file_name):
    with open(file_name,"w") as file:
        global doc_id_list_mapping
        for query_num in store_result_for_each_query:
            rank = 1
            for docid, score in store_result_for_each_query[query_num]:
                temp_str = query_num + " Q0 " + str(doc_id_list_mapping[docid][0]) + " " + str(rank) + " " +  str(score) + " " + "Exp"
                file.write(temp_str)
                file.write("\n")
                rank += 1
       

In [51]:
## Call to save each model's result

save_result (okapi_result, ".\data5\okapi")
save_result (tfidf_result, ".\data5\okapi_tfidf")
save_result (okapi_bm25_result, ".\data5\okapi_bm25")
save_result (laplace_result, ".\data5\laplace")
save_result (jm_result, ".\data5\jm")

In [68]:
## Get updated query

with open(r".\data\updated_query.txt", "r", encoding = "utf8") as file:
    queries = file.readlines()

new_updated_query = {(re.match("\d+",i[:4]).group()): i[3:] for i in queries}
new_updated_query_dict = {i : [(ps.stem(j)).lower() for j in [match.group().lower() for match in re.finditer(pattern,new_updated_query[i],re.M|re.I)]] for i in new_updated_query}
proximity_result = {}



In [160]:
## Search es for retrieving doc for each term in a query

dict_ids_prox = {}
for key in new_updated_query_dict:
    list_id = []
    t1 = time.time()
    for word in new_updated_query_dict[key]:
        try:
            term_id = flatted_dict[word]
        except KeyError:
            continue
        val = open_file_and_read_data(term_id = term_id, filename = new_filename, catalog = new_catalog)
        list_of_docs_term_present = list(val.values())
        list_of_docs_term_present_3000 = sorted(list_of_docs_term_present[0], key = lambda x:x[1],reverse = True)[:5000]
        list_of_docs_term_present_ids = [i[0] for i in list_of_docs_term_present_3000]
        list_id.extend(list_of_docs_term_present_ids)
    dict_ids_prox[key] = list(set(list_id))
    print ("Word completed:", key," time taken : ", (time.time() - t1), " for ids ", len(dict_ids_prox[key]))
    


Word completed: 85  time taken :  0.17756414413452148  for ids  6705
Word completed: 59  time taken :  0.04033541679382324  for ids  7245
Word completed: 56  time taken :  0.057399749755859375  for ids  9023
Word completed: 71  time taken :  0.08496594429016113  for ids  10102
Word completed: 64  time taken :  0.07426977157592773  for ids  7614
Word completed: 62  time taken :  0.11919069290161133  for ids  9828
Word completed: 93  time taken :  0.19465184211730957  for ids  10494
Word completed: 99  time taken :  0.02768564224243164  for ids  7225
Word completed: 58  time taken :  0.047683000564575195  for ids  9009
Word completed: 77  time taken :  0.0  for ids  2088
Word completed: 54  time taken :  0.04686880111694336  for ids  8921
Word completed: 87  time taken :  0.11591172218322754  for ids  12803
Word completed: 94  time taken :  0.04264497756958008  for ids  6512
Word completed: 100  time taken :  0.07690787315368652  for ids  10222
Word completed: 89  time taken :  0.1653497

In [161]:
# (C - rangeOfWindow) * numOfContainTerms / (lengthOfDocument + V)

# where C is a parameter which can be set to 1500 (you can play with C to achieve better results).

# rangeOfWindow refers to the output from the algorithm - http://stevekrenzel.com/articles/blurbs

# numOfContainTerms refers to the number of query terms contained by the document.

# lengthOfDocument - document length

# V - vocabulary size

def proximity_search_model(list_of_positions, doc_id, num_query_terms_contained_by_doc):
    C = 1500
    range_of_window = prox_search(list_of_positions)   
    prox_result =  ((C - range_of_window) * num_query_terms_contained_by_doc)/(len_of_doc["length"][doc_id] + vocab_size)
    if doc_id % 10000 == 0:
        print (list_of_positions,range_of_window, doc_id, num_query_terms_contained_by_doc, len_of_doc["length"][doc_id], prox_result)
    return prox_result


In [162]:
## Blurb algorithm

def prox_search(list_of_positions):
    len_l = len(list_of_positions)
    temp_dict = {i:False for i in range(len(list_of_positions))}
    total_trials = sum([len(j) for j in list_of_positions])
    range_val = float("inf")
    ## Initialize value_list with only the first element of each position list
    val_list = [list_of_positions[key][0] for key in range (len(list_of_positions))]
    if len_l == 1:
        return 0
    while (True):
        temp_val_list = [i for i in temp_dict if temp_dict[i] == False]
        if len(temp_val_list) == 1:
            break   
        increment_min = float("inf")
        for j in temp_val_list:
            if val_list[j] < increment_min:
                min_pointer = j
                increment_min =  val_list[j]
        current_max = max(val_list)
        current_min = min(val_list)
        if range_val > (current_max - current_min) and current_max !=  float("-inf"):
            range_val = current_max - current_min   

        current_i = list_of_positions[min_pointer].index(increment_min)
        try:
            increment_val = list_of_positions[min_pointer][current_i + 1]
            val_list[min_pointer] = increment_val
        except IndexError:
            temp_dict[min_pointer] = True
            
    return range_val

In [163]:
store_result_for_each_query = {}
dict_of_positions_present = {}
for key in new_updated_query_dict:
    dict_of_positions_present[key] = []
    for word in new_updated_query_dict[key]:
        term_id = flatted_dict[word]
        val = open_file_and_read_data(term_id = term_id, filename = new_filename, catalog = new_catalog)
        dict_ = {i[0]:i[2:] for i in list(val.values())[0] if i[0] in dict_ids_prox[key]}
        dict_of_positions_present[key].extend(sorted(dict_.items(), key = lambda x:len(x[1]), reverse = True))
    #dict_of_positions = [i for j in dict_of_positions_present for i in dict_of_positions_present[j]]
    sorted_v = sorted(dict_of_positions_present[key], key = lambda x:x[0])
    another_dict = {}
    i = 0
    while(i < len(sorted_v) - 2):
        no_val = [(len_of_doc["length"][sorted_v[i][0]])*4]
        if sorted_v[i][0] == sorted_v[i+1][0]:
            if sorted_v[i+1][0] == sorted_v[i+2][0]:
                another_dict[sorted_v[i][0]] = [sorted_v[i][1], sorted_v[i+1][1], sorted_v[i+2][1]]
                i += 3
            else:
                another_dict[sorted_v[i][0]] = [sorted_v[i][1], sorted_v[i+1][1], no_val]
                i += 2
        else:
            another_dict[sorted_v[i][0]] = [sorted_v[i][1], no_val, [no_val[0] * 2]]
            i += 1        
    result_dict = {}
    for doc_id in another_dict:
        no_val = (len_of_doc["length"][doc_id])*4
        
        num_query_terms_contained_by_doc = [i for i in another_dict[doc_id] if i not in [[no_val],[no_val * 2]]]
        result_dict[doc_id] = proximity_search_model(another_dict[doc_id], doc_id, len(num_query_terms_contained_by_doc))
    store_result_for_each_query[key] = sorted(result_dict.items(), key = lambda x:x[1], reverse = True)[:1000]
    
save_result (store_result_for_each_query, ".\data5\prox_temp")    


[[18, 159, 198], [1160], [2320]] 2122 10000 1 290 -0.004014871808112365
[[22, 87, 94, 102], [720], [1440]] 1338 30000 1 180 0.0010464169907114344
[[14, 31, 41, 202, 257], [1068], [2136]] 1879 40000 1 267 -0.002446724036642759
[[29], [1160], [2320]] 2291 10000 1 290 -0.005105729260798843
[[6], [864], [1728]] 1722 60000 1 216 -0.0014336454633516306
[[242, 254], [1112], [2224]] 1970 50000 1 278 -0.003033980582524272
[[105], [736], [1472]] 1367 70000 1 184 0.0008590732343784314
[[13, 79, 92, 104, 147, 174, 186, 217, 227, 238], [1160], [2320]] 2082 10000 1 290 -0.0037566806950504765
[[105], [736], [1472]] 1367 70000 1 184 0.0008590732343784314
[[99], [736], [1472]] 1373 70000 1 184 0.0008203180508726376
[[7, 48], [720], [1440]] 1392 30000 1 180 0.0006976113271409563
[[199], [1068], [2136]] 1937 40000 1 267 -0.002821156738820279
[[221], [1068], [2136]] 1915 40000 1 267 -0.0026791305414425987
[[34], [1112], [2224]] 2190 50000 1 278 -0.004454141706259037
[[74], [736], [1472]] 1398 70000 1 184 

In [164]:
# ## change not present terms
# ## Call proximity search and store the result

# store_result_for_each_query = {}
# dict_of_positions_present = {}
# for key in new_updated_query_dict:
#     dict_of_positions_present[key] = []
#     for word in new_updated_query_dict[key]:
#         try:
#             term_id = flatted_dict[word]
#         except:
#             continue
#         val = open_file_and_read_data(term_id = term_id, filename = new_filename, catalog = new_catalog)
#         dict_ = {i[0]:i[2:] for i in list(val.values())[0] if i[0] in dict_ids_prox[key]}
#         dict_of_positions_present[key].extend(sorted(dict_.items(), key = lambda x:len(x[1]), reverse = True))
#     sorted_v = sorted(dict_of_positions_present[key], key = lambda x:x[0])
#     another_dict = {i[0]:[] for i in sorted_v}
#     for i in sorted_v:
#         another_dict[i[0]].append(i[1])        
#         result_dict = {}
#     for doc_id in another_dict:
#         result_dict[doc_id] = proximity_search_model(another_dict[doc_id], doc_id, len(another_dict[doc_id]))
#     store_result_for_each_query[key] = sorted(result_dict.items(), key = lambda x:x[1], reverse = True)[:1000]
    
# save_result (store_result_for_each_query, ".\data4\prox4")    


In [178]:
!jupyter nbconvert --to python HW2-IR.ipynb

[NbConvertApp] Converting notebook HW2-IR.ipynb to python
[NbConvertApp] Writing 23480 bytes to HW2-IR.py
